# Assignment 3


#### Load our Python tools

In [1]:
# %load_ext lab_black

In [2]:
import pandas as pd
import altair as alt

In [3]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

---

#### Import statewide dataset

In [4]:
covid_data=pd.read_csv("https://raw.githubusercontent.com/datadesk/california-coronavirus-data/master/cdph-state-cases-deaths.csv").sort_values('date')

#### Data are cumulative, so we need to extract how many new cases there were each day

In [5]:
covid_data['daily_new_cases'] = covid_data['confirmed_cases'].diff().fillna(covid_data['confirmed_cases'])

In [6]:
covid_data.head()

,date,confirmed_cases,reported_cases,probable_cases,reported_and_probable_cases,reported_deaths,daily_new_cases
765,2020-02-01,23,3,1,4,0,23.0
764,2020-02-02,30,6,1,7,0,7.0
763,2020-02-03,35,6,1,7,0,5.0
762,2020-02-04,35,6,2,8,0,0.0
761,2020-02-05,38,6,2,8,0,3.0


In [7]:
covid_data.tail()

,date,confirmed_cases,reported_cases,probable_cases,reported_and_probable_cases,reported_deaths,daily_new_cases
4,2022-03-03,8411558,8399677,587255,8986932,85353,2504.0
3,2022-03-04,8413279,8399677,587255,8986932,85353,1721.0
2,2022-03-05,8413944,8399677,587255,8986932,85353,665.0
1,2022-03-06,8414099,8412311,587255,8999566,85849,155.0
0,2022-03-07,8414099,8414669,587255,9001924,85854,0.0


#### Create a generic month column for aggregation

In [8]:
covid_data["month"]=pd.to_datetime(covid_data["date"]).dt.strftime("%Y-%m-01")

In [9]:
covid_data.head()

,date,confirmed_cases,reported_cases,probable_cases,reported_and_probable_cases,reported_deaths,daily_new_cases,month
765,2020-02-01,23,3,1,4,0,23.0,2020-02-01
764,2020-02-02,30,6,1,7,0,7.0,2020-02-01
763,2020-02-03,35,6,1,7,0,5.0,2020-02-01
762,2020-02-04,35,6,2,8,0,0.0,2020-02-01
761,2020-02-05,38,6,2,8,0,3.0,2020-02-01


#### Group by month to sum new cases by month

In [10]:
covid_group_months = covid_data.groupby(["month"]).agg({"daily_new_cases":"sum"}).reset_index()

#### How many new cases per month? 

In [11]:
covid_group_months.rename(columns={'daily_new_cases':'new_monthly_cases'}, inplace=True)

In [12]:
covid_group_months.head()

,month,new_monthly_cases
0,2020-02-01,466.0
1,2020-03-01,19273.0
2,2020-04-01,46543.0
3,2020-05-01,71247.0
4,2020-06-01,174058.0


In [13]:
covid_group_months["new_monthly_covid_cases_per_100k"]=((covid_group_months.new_monthly_cases/38000000)*100000).round(1)

In [14]:
covid_group_months

,month,new_monthly_cases,new_monthly_covid_cases_per_100k
0,2020-02-01,466.0,1.2
1,2020-03-01,19273.0,50.7
2,2020-04-01,46543.0,122.5
3,2020-05-01,71247.0,187.5
4,2020-06-01,174058.0,458.0
5,2020-07-01,280834.0,739.0
6,2020-08-01,149064.0,392.3
7,2020-09-01,98748.0,259.9
8,2020-10-01,120594.0,317.4
9,2020-11-01,399997.0,1052.6


#### Now that you have Altair!

In [15]:
#### Omicron pops!

In [16]:
alt.Chart(covid_group_months).mark_bar(size=10).encode(
    x=alt.X('month:T', axis=alt.Axis(format="%b. %y")),
    y='new_monthly_cases:Q'
).properties(title='New CA COVID cases by month')

alt.Chart(...)

---

#### Merge with flights

Good luck!!

In [17]:
flight_data=pd.read_csv("data/Los_Angeles_International_Airport_-_Flight_Operations_By_Month.csv", dtype = {"DataExtractDate":str})

In [18]:
flight_data["clean_date"]= pd.to_datetime(flight_data["ReportPeriod"]).dt.strftime("%Y-%m-%d")

In [19]:
flight_data.head()

,DataExtractDate,ReportPeriod,FlightType,Arrival_Departure,Domestic_International,FlightOpsCount,clean_date
0,04/01/2021 09:37:48 AM,02/01/2021 12:00:00 AM,Scheduled,Departure,International,1690,2021-02-01
1,04/18/2018 03:12:41 PM,12/01/2017 12:00:00 AM,Scheduled,Departure,Domestic,20797,2017-12-01
2,04/18/2018 03:12:41 PM,10/01/2008 12:00:00 AM,Charter,Departure,International,9,2008-10-01
3,04/18/2018 03:12:41 PM,02/01/2016 12:00:00 AM,Scheduled,Arrival,Domestic,18427,2016-02-01
4,04/18/2018 03:12:41 PM,09/01/2007 12:00:00 AM,Scheduled,Departure,International,4227,2007-09-01


In [20]:
flight_data[flight_data["clean_date"]=="2018-04-17"]

,DataExtractDate,ReportPeriod,FlightType,Arrival_Departure,Domestic_International,FlightOpsCount,clean_date


In [21]:
flight_ops_data=flight_data[flight_data["Arrival_Departure"]=="Departure"].groupby(["clean_date"])["FlightOpsCount"].sum()

In [22]:
flight_ops_data

clean_date
2006-01-01    23844
2006-02-01    21394
2006-03-01    24222
2006-04-01    23327
2006-05-01    24279
2006-06-01    24276
2006-07-01    25534
2006-08-01    25790
2006-09-01    23638
2006-10-01    24455
2006-11-01    23436
2006-12-01    24808
2007-01-01    24693
2007-02-01    22125
2007-03-01    25165
2007-04-01    24101
2007-05-01    24764
2007-06-01    24936
2007-07-01    27098
2007-08-01    27226
2007-09-01    25123
2007-10-01    25961
2007-11-01    25121
2007-12-01    26099
2008-01-01    24710
2008-02-01    22797
2008-03-01    24401
2008-04-01    23721
2008-05-01    24473
2008-06-01    25021
2008-07-01    25840
2008-08-01    25304
2008-09-01    21162
2008-10-01    21888
2008-11-01    19929
2008-12-01    20942
2009-01-01    20520
2009-02-01    18640
2009-03-01    21182
2009-04-01    20496
2009-05-01    21021
2009-06-01    21436
2009-07-01    22909
2009-08-01    22682
2009-09-01    20468
2009-10-01    21349
2009-11-01    20427
2009-12-01    21597
2010-01-01    21269
2010-02-0

In [23]:
covid_flight_merge=pd.merge(covid_data, flight_ops_data, on = ["DataExtractDate"])

KeyError: 'DataExtractDate'

In [ ]:
covid_flight_merge.head()